In [281]:
import pandas as pd
import seaborn as sns
import datetime
from datetime import *
import sys
import gzip
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pylab
import scipy.stats as stats
import plotly.graph_objects as go
import collections
from scipy.stats import bootstrap, ttest_ind, ttest_rel


from sklearn.cluster import KMeans

In [282]:
plt.style.use('seaborn')
pd.options.mode.chained_assignment = None  # default='warn'

# beer

In [283]:
#load the data beers.csv
df_beers = pd.read_csv('./data/beers.csv')

In [284]:
df_beers.head()

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.7,NaN,NaN,0,NaN
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,3.8,NaN,NaN,0,NaN
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.8,NaN,NaN,0,NaN
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.0,NaN,NaN,0,NaN
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.5,NaN,NaN,0,NaN


In [285]:
list_considered_variable=['beer_id','beer_name','brewery_id','brewery_name','style',
                          'nbr_reviews','nbr_ratings','avg', 'abv']
df_beers=df_beers[list_considered_variable]

In [286]:
beer_catalog = {
    'Bocks' : ['Bock', 'Doppelbock', 'Eisbock', 'Maibock', 'Weizenbock', 'Maibock / Helles Bock'],

    'Dark Ales' : ['Dubbel', 'Roggenbier', 'Scottish Ale', 'Winter Warmer'],

    'Hybrid Beers' : ['Bière de Champagne / Bière Brut', 'Braggot', 'California Common / Steam Beer', 'Cream Ale'],

    'Pale Ales' : ['American Amber / Red Ale', 'American Blonde Ale', 'American Pale Ale', 'Belgian Blonde Ale',
    'Belgian Pale Ale', 'Bière de Garde', 'English Bitter', 'English Pale Ale', 'English Pale Mild Ale', 'Extra Special / Strong Bitter (ESB)',
    'Grisette', 'Irish Red Ale', 'Kölsch', 'Saison', 'Saison / Farmhouse Ale', 'Belgian Strong Pale Ale', 'American Pale Ale (APA)'],

    'Porters' : ['American Porter', 'Baltic Porter', 'English Porter', 'Imperial Porter', 'Robust Porter ', 'Smoked Porter'],

    'Stouts' : ['American Imperial Stout', 'American Stout', 'English Stout', 'Foreign / Export Stout', 'Irish Dry Stout', 'Oatmeal Stout',
    'Russian Imperial Stout', 'Milk / Sweet Stout', 'American Double / Imperial Stout'],

    'Wheat Beers' : ['American Dark Wheat Beer', 'American Pale Wheat Beer', 'Dunkelweizen', 'Grodziskie', 'Hefeweizen', 'Kristallweizen', 'Witbier', 
    'American Pale Wheat Ale', 'American Dark Wheat Ale', 'Berliner Weissbier', 'Kristalweizen'],

    'Brown Ales' : ['Altbier', 'American Brown Ale', 'Belgian Dark Ale', 'English Brown Ale', 'English Dark Mild Ale', 'Belgian Strong Dark Ale'],

    'Dark Lagers' : ['American Amber / Red Lager', 'Czech Amber Lager', 'Czech Dark Lager', 'European Dark Lager', 'Märzen',
    'Munich Dunkel', 'Rauchbier', 'Schwarzbier', 'Vienna Lager', 'Munich Dunkel Lager', 'Euro Dark Lager'],

    'India Pale Ales' : ['American IPA', 'American Double / Imperial IPA', 'Belgian IPA', 'Black IPA', 'Brut IPA', 'English IPA', 'English India Pale Ale (IPA)', 
    'Imperial IPA', 'Milkshake IPA', 'New England IPA', 'American Black Ale'],

    'Pale Lagers' : ['American Adjunct Lager', 'American Lager', 'Bohemian / Czech Pilsner', 'Czech Pale Lager', 'European / Dortmunder Export Lager',
    'European Pale Lager', 'European Strong Lager', 'Festbier / Wiesnbier', 'German Pilsner', 'Helles', 'Imperial Pilsner', 'India Pale Lager (IPL)',
    'Kellerbier / Zwickelbier', 'Light Lager', 'Malt Liquor', 'Czech Pilsener', 'Euro Pale Lager', 'American Pale Lager', 'German Pilsener', 'Dortmunder / Export Lager', 
    'Euro Strong Lager', 'Munich Helles Lager', 'American Double / Imperial Pilsner', 'American Malt Liquor', 'Märzen / Oktoberfest'],

    'Specialty Beer' : ['Chile Beer', 'Fruit and Field Beer', 'Gruit / Ancient Herbed Ale', 'Happoshu', 'Herb and Spice Beer', 'Japanese Rice Lager',
    'Kvass', 'Low-Alcohol Beer', 'Pumpkin Beer', 'Rye Beer', 'Sahti', 'Smoked Beer', 'Herbed / Spiced Beer', 'Pumpkin Ale', 'Scottish Gruit / Ancient Herbed Ale', 'Low Alcohol Beer'],

    'Strong Ales' : ['American Barleywine', 'American Strong Ale', 'Belgian Dark Strong Ale', 'Belgian Pale Strong Ale', 'English Barleywine', 'English Strong Ale',
    'Imperial Red Ale', 'Old Ale', 'Quadrupel (Quad)', 'Scotch Ale / Wee Heavy', 'Tripel', 'Wheatwine', 'Fruit / Vegetable Beer'],
    
    'Wild/Sour Beers' : ['Berliner Weisse', 'Brett Beer', 'Faro', 'Flanders Oud Bruin', 'Flanders Red Ale', 'Fruit Lambic',
    'Fruited Kettle Sour', 'Gose', 'Gueuze', 'Lambic', 'Wild Ale', 'Lambic - Unblended', 'Lambic - Fruit', 'American Wild Ale'],

    'Weird Cocktail' : ['Black & Tan']
}

In [287]:
beer_catalog_reverse = {}
for key, values in beer_catalog.items():
    for value in values:
            beer_catalog_reverse.setdefault(value, []).append(key)

In [288]:
def category_check(beer_style, beer_catalog):
    '''
    Return the new columns of 15 categories made in the website https://www.beeradvocate.com/beer/styles/
    '''
    for category_name, beer_styles in beer_catalog.items():
        if beer_style in beer_styles:
            return category_name

In [289]:
df_beers['category'] = df_beers['style'].apply(lambda beer_style: category_check(beer_style, beer_catalog))

In [290]:
df_beers

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_reviews,nbr_ratings,avg,abv,category
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.7,Pale Lagers
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,3.8,Pale Lagers
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.8,Pale Lagers
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.0,Pale Lagers
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.5,Pale Lagers
...,...,...,...,...,...,...,...,...,...,...
280818,19139,Kölsch Ale,885,Summit Station Restaurant & Brewery,Kölsch,3,3,2.71,5.0,Pale Ales
280819,19140,Nut Brown Ale,885,Summit Station Restaurant & Brewery,English Brown Ale,1,2,3.10,5.3,Brown Ales
280820,19146,Octoberfest,885,Summit Station Restaurant & Brewery,Märzen / Oktoberfest,0,0,NaN,NaN,Pale Lagers
280821,2805,Scotch Ale,885,Summit Station Restaurant & Brewery,Scotch Ale / Wee Heavy,0,0,NaN,7.0,Strong Ales


# Users and filtering

In [291]:
#load the data

df_users = pd.read_csv('./data/users.csv')
print(df_users.describe())
df_users.head(5)

         nbr_ratings    nbr_reviews        joined
count  153704.000000  153704.000000  1.510520e+05
mean       54.605163      16.847876  1.357238e+09
std       252.388790     139.846706  9.195131e+07
min         1.000000       0.000000  8.407944e+08
25%         1.000000       0.000000  1.303121e+09
50%         3.000000       0.000000  1.391944e+09
75%        16.000000       2.000000  1.417691e+09
max     12046.000000    8970.000000  1.501495e+09


,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
0,7820,465,nmann08.184925,nmann08,1.199704e+09,"United States, Washington"
1,2521,2504,stjamesgate.163714,StJamesGate,1.191838e+09,"United States, New York"
2,1797,1143,mdagnew.19527,mdagnew,1.116410e+09,Northern Ireland
3,31,31,helloloser12345.10867,helloloser12345,1.101380e+09,Northern Ireland
4,604,604,cypressbob.3708,cypressbob,1.069326e+09,Northern Ireland


In [292]:
# First user filtering phase 

# keep users from the USA only & who have made a minimum number of reviews in order to participate in our study

df_us_users = df_users[df_users.location.str.contains('United States', na=False)]
nb_us_users = df_us_users['user_id'].nunique()
print(f"There are {nb_us_users} US users")

#choose a minimum number of ratings per user, since we want to analyse an evolution!
cutoff_low = 500
print("Cutoff low : ", cutoff_low)

#choose a maximum number of ratings per user, we want to filter our super-users or organisations
cutoff_high = df_us_users.nbr_ratings.quantile(0.99)
print("Cutoff high : ", cutoff_high)

#create a valid user dataframe and a set with the valid user_names
df_us_users = df_us_users[(df_us_users.nbr_ratings < cutoff_high) &
                                (df_us_users.nbr_ratings >= cutoff_low)]
us_users_names = set(df_us_users.user_name)
print("Number of users left : ", len(us_users_names))
print("Total of ratings left : ", df_us_users.nbr_ratings.sum())

#convert the 'joined' column into datetime format
df_us_users.loc[:,'date_joined'] = pd.to_datetime(df_us_users['joined'].apply(lambda dt : datetime.fromtimestamp(dt).strftime('%Y-%m-%d')))

There are 113223 US users
Cutoff low :  500
Cutoff high :  1135.0
Number of users left :  2232
Total of ratings left :  1652164


In [293]:
# We now gather some more user information by analyzing their behaviour as a reviewer

In [294]:
def parse(filename):
    """
    Parse a txt.gz file and return a generator for it
    Copyright © 2017 Gael Lederrey <gael.lederrey@epfl.ch>
    :param filename: name of the file
    :return: Generator to go through the file
    """
    file = gzip.open(filename, 'rb')
    entry = {}
    # Go through all the lines
    for line in file:
        # Transform the string-bytes into a string
        line = line.decode("utf-8").strip()

        # We check for a colon in each line
        colon_pos = line.find(":")
        if colon_pos == -1:
            # if no, we yield the entry
            yield entry
            entry = {}
            continue
        # otherwise, we add the key-value pair to the entry
        key = line[:colon_pos]
        value = line[colon_pos + 2:]
        entry[key] = value

In [295]:
gen = parse('./data/ratings.txt.gz')

In [296]:
#We create the dataframe of the valid us users previously determined

create = False

if create:
    undesired_features = set(['text','review', 'brewery_name', 'brewery_id', 'user_id', 'beer_name'])
    full_ratings_ls = []
    int_cols = ['beer_id']
    float_cols = ['abv','appearance', 'aroma', 'palate', 'taste', 'overall', 'rating']

    review = next(gen)
    cols = [key for key, _ in review.items() if key not in undesired_features]

    for idx, review in enumerate(gen):
        user_name = review['user_name']
        valid_rows = [value for key, value in review.items() if key not in undesired_features]
        if user_name in us_users_names:
            full_ratings_ls.append(valid_rows)

    df_ratings = pd.DataFrame(full_ratings_ls, columns = cols)

    df_ratings[int_cols] = df_ratings[int_cols].astype('int')
    df_ratings[float_cols] = df_ratings[float_cols].astype('float') 
    df_ratings["date"] = pd.to_datetime(df_ratings.date.apply(
        lambda date : datetime.fromtimestamp(int(date)).strftime('%Y-%m-%d')
    ))

    #There is one review in 2000, wich is an outlier
    start_date = pd.to_datetime("2001-01-01")
    df_ratings = df_ratings.loc[df_ratings['date'] >= start_date]   
    df_ratings.to_pickle('./data/ratings_valid_user.pkl')

else:
    df_ratings = pd.read_pickle('./data/ratings_valid_user.pkl')

In [297]:
#add the category of the beer
beers_id_cat = df_beers[['beer_id', 'category']]
df_ratings = pd.merge(df_ratings, beers_id_cat, on = 'beer_id')

#add the rating number, starts at 0
df_ratings.sort_values(by = "date", inplace=True)
df_ratings["rating_number"] = df_ratings.groupby("user_name").transform("cumcount")

#we also convert the date to the proper format
df_ratings["date"] = pd.to_datetime(df_ratings["date"])

In [298]:
#We apply the second of our filters, we only keep the 500 first ratings. (rating number at which you are considered an expert)
nb_ratings_expert = 500
df_us_ratings = df_ratings[df_ratings["rating_number"] < nb_ratings_expert]

In [299]:
df_us_ratings.head(5)

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number
1640018,167,Witbier,NaN,2001-03-16,frank4sail,4.5,4.5,4.0,5.0,5.0,4.75,Wheat Beers,0
311428,3558,Czech Pilsener,5.00,2001-06-18,Mark,4.0,4.0,3.0,4.0,4.0,3.90,Pale Lagers,0
17313,73,Milk / Sweet Stout,5.20,2001-06-18,Mark,5.0,5.0,5.0,5.0,5.0,5.00,Stouts,1
136697,261,Vienna Lager,4.48,2001-06-24,Mark,4.0,3.0,3.0,3.0,3.0,3.06,Dark Lagers,2
442671,87,American Pale Ale (APA),5.40,2001-06-24,Mark,4.0,4.0,1.0,3.0,3.0,3.10,Pale Ales,3


In [300]:
#We go back to users and compute on how many different days they were active

user_active = df_us_ratings.groupby(['user_name'], as_index = False).date.nunique()
user_active.columns = ['user_name', 'active_days']
df_us_users = df_us_users.merge(user_active, on = 'user_name')

In [301]:
# We compute how long it took them to submit the 500 ratings
user_delta = df_us_ratings.groupby('user_name', as_index = False).apply(lambda usr: (usr.date.max() - usr.date.min()).days)
user_delta.columns = ['user_name', 'delta_first_last']
df_us_users = df_us_users.merge(user_delta, on = 'user_name')
df_us_users

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location,date_joined,active_days,delta_first_last
0,1131,188,okcnittany.144868,OKCNittany,1.182766e+09,"United States, Oklahoma",2007-06-25,187,2397
1,732,14,byofb.537281,BYOFB,1.292065e+09,"United States, Wisconsin",2010-12-11,32,756
2,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois",2009-04-15,202,1193
3,1078,10,colsanders123.545441,ColSanders123,1.293620e+09,"United States, Georgia",2010-12-29,135,1077
4,914,890,jays2629.517433,JayS2629,1.287828e+09,"United States, Alabama",2010-10-23,101,244
...,...,...,...,...,...,...,...,...,...
2227,548,21,crafteyedoc.675112,CraftEyeDoc,1.336039e+09,"United States, North Carolina",2012-05-03,143,970
2228,630,23,pmoney.592770,pmoney,1.302862e+09,"United States, Illinois",2011-04-15,132,854
2229,637,48,aleckpa.541457,aleckpa,1.292843e+09,"United States, North Carolina",2010-12-20,275,1853
2230,1117,4,dinoblueshoes.607196,DinoBlueShoes,1.309342e+09,"United States, Maine",2011-06-29,196,1047


In [302]:
# We compute the maximum number of ratings the user has done in a single day

user_day = df_us_ratings[['user_name','date']].groupby(["user_name", pd.Grouper(freq = "D", key="date")], as_index = False).size()
user_day = user_day.groupby('user_name', as_index = False).apply(lambda usr : max(usr['size']))
user_day.columns = ['user_name', 'max_active_day']
df_us_users = df_us_users.merge(user_day, on = 'user_name')

In [303]:
# We compute the maximum number of ratings the user has done in a single month

user_month = df_us_ratings[['user_name','date']].groupby(["user_name", pd.Grouper(freq = "M", key="date")], as_index = False).size()
user_month = user_month.groupby('user_name', as_index = False).apply(lambda usr : max(usr['size']))
user_month.columns = ['user_name', 'max_active_month']
df_us_users = df_us_users.merge(user_month, on = 'user_name') 

In [304]:
# We also want to know how much time the users took for submitting different number of ratings.

def compute_diff_date(user):
    diff = {}
    day = 50
    diff[str(day)] = (user[user["rating_number"] == day-1]["date"].item() -\
                         user[user["rating_number"] == 0]["date"].item()).days
    return pd.Series(diff)

In [305]:
def compute_diff_date_last(user):
    diff = {}
    day = 450
    diff[str(day)] = (user[user["rating_number"] == 500-1]["date"].item() -\
                         user[user["rating_number"] == day]["date"].item()).days
    return pd.Series(diff)

In [306]:
# We focus on the 10% first reviews (more justification)
user_deltas_0_50 = df_us_ratings.groupby('user_name', as_index = False).apply(compute_diff_date)
df_us_users = df_us_users.merge(user_deltas_0_50, on = 'user_name')

In [307]:
# We focus on the 10% last reviews (more justification)
user_deltas_450_500 = df_us_ratings.groupby('user_name', as_index = False).apply(compute_diff_date_last)
df_us_users = df_us_users.merge(user_deltas_450_500, on = 'user_name')

In [308]:
df_us_users.head(5)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location,date_joined,active_days,delta_first_last,max_active_day,max_active_month,50,450
0,1131,188,okcnittany.144868,OKCNittany,1.182766e+09,"United States, Oklahoma",2007-06-25,187,2397,11,91,45,16
1,732,14,byofb.537281,BYOFB,1.292065e+09,"United States, Wisconsin",2010-12-11,32,756,232,273,447,146
2,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois",2009-04-15,202,1193,12,42,64,335
3,1078,10,colsanders123.545441,ColSanders123,1.293620e+09,"United States, Georgia",2010-12-29,135,1077,238,247,254,159
4,914,890,jays2629.517433,JayS2629,1.287828e+09,"United States, Alabama",2010-10-23,101,244,20,103,21,15


In [309]:
# We now have a lot more information about the reviewing behaviour of our us users
# We proceed with some more filtering to get the final set of users on which we will conduct our study.

# Minimum number of active days required:
min_active_days = 100
df_valid_users = df_us_users[df_us_users['active_days'] >= min_active_days]

# How many days needed to submit their 500 ratings:
min_delta = 365
max_delta = 7*365
df_valid_users = df_valid_users[(df_valid_users['delta_first_last'] > min_delta) & (df_valid_users['delta_first_last'] < max_delta)]
# Maximum number of reviews in a single day: 
# On utilise pas pour le moment
max_per_day = 70

#df_valid_users = df_valid_users[df_valid_users['max_active_day'] < 70]
# Maximum number of reviews in a single month:
max_per_month = 70
df_valid_users = df_valid_users[df_valid_users['max_active_month'] <= max_per_month]

In [310]:
# To make the people more similar at the beginning and the end of their ratings career, 
# we decide to apply a last filter on the users

In [311]:
df_valid_users.head(5)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location,date_joined,active_days,delta_first_last,max_active_day,max_active_month,50,450
2,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois",2009-04-15,202,1193,12,42,64,335
5,676,676,dodo2step.110880,Dodo2step,1.165576e+09,"United States, Pennsylvania",2006-12-08,411,2153,13,28,45,442
7,653,651,kadonny.156863,Kadonny,1.188986e+09,"United States, Pennsylvania",2007-09-05,357,1800,5,22,249,263
8,748,748,biggred1.206640,biggred1,1.206616e+09,"United States, Indiana",2008-03-27,318,819,5,37,102,69
9,546,484,brownbeer.220672,brownbeer,1.211105e+09,"United States, Missouri",2008-05-18,143,1815,17,62,204,1302


In [312]:
df_check = df_valid_users['50'].to_frame()
df_check.sort_values(by = '50', inplace=True)
fig = px.line(df_check, x = range(df_check.shape[0]), y = '50')
fig.add_hline(500, line_color = 'red')
fig.show()

In [313]:
max_time_to_rate = 500
df_valid_users = df_valid_users[(df_valid_users['50'] < max_time_to_rate)]

In [314]:
df_check = df_valid_users['450'].to_frame()
df_check.sort_values(by = '450', inplace=True)
fig = px.line(df_check, x = range(df_check.shape[0]), y = '450')
fig.add_hline(500, line_color = 'red')
fig.show()

In [315]:
# We recut at 500
max_time_to_rate = 500
df_valid_users = df_valid_users[(df_valid_users['450'] < max_time_to_rate)]

In [316]:
# We have the final data set of users and the data set of their ratings

valid_usernames = set(df_valid_users['user_name'])
df_valid_ratings = df_us_ratings[df_us_ratings['user_name'].isin(valid_usernames)]

In [317]:
df_valid_users.head(5)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location,date_joined,active_days,delta_first_last,max_active_day,max_active_month,50,450
2,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois",2009-04-15,202,1193,12,42,64,335
5,676,676,dodo2step.110880,Dodo2step,1.165576e+09,"United States, Pennsylvania",2006-12-08,411,2153,13,28,45,442
7,653,651,kadonny.156863,Kadonny,1.188986e+09,"United States, Pennsylvania",2007-09-05,357,1800,5,22,249,263
8,748,748,biggred1.206640,biggred1,1.206616e+09,"United States, Indiana",2008-03-27,318,819,5,37,102,69
10,614,614,whynot44.2033,whynot44,1.052129e+09,"United States, Missouri",2003-05-05,276,1452,7,57,24,316


In [318]:
df_valid_ratings.head(5)

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number
1640018,167,Witbier,NaN,2001-03-16,frank4sail,4.5,4.5,4.0,5.0,5.0,4.75,Wheat Beers,0
311428,3558,Czech Pilsener,5.00,2001-06-18,Mark,4.0,4.0,3.0,4.0,4.0,3.90,Pale Lagers,0
17313,73,Milk / Sweet Stout,5.20,2001-06-18,Mark,5.0,5.0,5.0,5.0,5.0,5.00,Stouts,1
136697,261,Vienna Lager,4.48,2001-06-24,Mark,4.0,3.0,3.0,3.0,3.0,3.06,Dark Lagers,2
442671,87,American Pale Ale (APA),5.40,2001-06-24,Mark,4.0,4.0,1.0,3.0,3.0,3.10,Pale Ales,3


In [319]:
# Clustering starts
# We want to check if it is possible to extract meaningful clusters
# out of the 50 first ratings submitted by users

In [320]:
# To do that, we check how many beers of different categories 
# a user rated in his first 50 ratings

df_to_cluster = df_valid_ratings[df_valid_ratings["rating_number"] < 50]
user_to_cat_first = df_to_cluster.groupby(["user_name", "category"]).size()
user_to_cat_first = user_to_cat_first.unstack(level=-1, fill_value = 0)
user_to_cat_first

category,Bocks,Brown Ales,Dark Ales,Dark Lagers,Hybrid Beers,India Pale Ales,Pale Ales,Pale Lagers,Porters,Specialty Beer,Stouts,Strong Ales,Weird Cocktail,Wheat Beers,Wild/Sour Beers
user_name,,,,,,,,,,,,,,,
-Rick,0,2,0,2,0,29,7,2,2,2,3,1,0,0,0
1000lbgrizzly,0,5,0,1,0,4,9,7,2,8,6,4,0,3,1
1424IpA,0,0,1,0,0,21,3,1,1,1,12,4,0,1,5
2Xmd,0,2,1,1,0,25,5,1,1,2,9,1,0,2,0
2ellas,0,2,1,0,0,31,9,0,0,0,2,4,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zenithberwyn,1,1,0,3,0,3,7,10,3,0,14,5,0,3,0
zhanson88,1,5,3,1,0,6,6,2,3,1,12,7,0,3,0
zookerman182,1,3,0,0,0,22,6,0,2,0,9,5,0,1,1


In [321]:
#Rename tu user_clustering?

our_clustering_4 = KMeans(4).fit(user_to_cat_first.values)
our_clustering_5 = KMeans(5).fit(user_to_cat_first.values)

In [322]:
count = collections.Counter(our_clustering_4.labels_)
count

Counter({0: 239, 3: 394, 1: 196, 2: 141})

In [323]:
#Display 4 clusters

df_cluster_beer = pd.DataFrame(our_clustering_4.cluster_centers_.T)
df_cluster_beer.index = user_to_cat_first.columns
df_cluster_beer["style"] = df_cluster_beer.index
df_cluster_beer = pd.melt(df_cluster_beer, id_vars=["style"], value_vars=[0,1,2,3])
df_cluster_beer.columns = ["style", "cluster", "Number of beer"]
df_cluster_beer

,style,cluster,Number of beer
0,Bocks,0,0.598326
1,Brown Ales,0,1.732218
2,Dark Ales,0,0.615063
3,Dark Lagers,0,0.548117
4,Hybrid Beers,0,0.234310
5,India Pale Ales,0,22.832636
6,Pale Ales,0,7.468619
7,Pale Lagers,0,2.012552
8,Porters,0,1.351464
9,Specialty Beer,0,1.221757


In [324]:
#display 5 clusters
df_cluster_beer_5 = pd.DataFrame(our_clustering_5.cluster_centers_.T)
df_cluster_beer_5.index = user_to_cat_first.columns
df_cluster_beer_5["category"] = df_cluster_beer_5.index
df_cluster_beer_5 = pd.melt(df_cluster_beer_5, id_vars=["category"], value_vars=[0,1,2,3,4])
df_cluster_beer_5.columns = ["category", "cluster", "Number of beer"]
df_cluster_beer_5

,category,cluster,Number of beer
0,Bocks,0,0.372263
1,Brown Ales,0,1.291971
2,Dark Ales,0,0.430657
3,Dark Lagers,0,0.489051
4,Hybrid Beers,0,0.197080
...,...,...,...
70,Stouts,4,5.411565
71,Strong Ales,4,4.982993
72,Weird Cocktail,4,0.040816
73,Wheat Beers,4,2.608844


In [325]:
fig = px.line(df_cluster_beer, x="style", y="Number of beer", color="cluster", title="Centroids obtained with Kmeans (k=4) of the first 50 ratings of each user")
fig.show()
#fig.write_html("centroid.html")

we see that 3 cluster are really corelatd -> can be merged into 2 : IPA lovers, and beer discoverer -> keep k = 4

In [326]:
df_valid_ratings.to_pickle('leandrinho.pkl')

In [327]:
fig = px.line(df_cluster_beer_5, x="category", y="Number of beer", color="cluster", title="Centroids obtained with Kmeans (k=5) of the first 50 ratings of each user")
fig.show()
#fig.write_html("centroid_first_5_justification.html")

In [328]:
user_to_cat_first["first_50_cluster"] = our_clustering_4.labels_

In [329]:
# How many beers of different categories a user rated in his last 50 ratings

df_to_cluster_last = df_valid_ratings[df_valid_ratings["rating_number"] >= 450]
user_to_cat_last = df_to_cluster_last.groupby(["user_name", "category"])["beer_id"].count()
user_to_cat_last = user_to_cat_last.unstack(level=-1, fill_value = 0)
user_to_cat_last.head(5)

category,Bocks,Brown Ales,Dark Ales,Dark Lagers,Hybrid Beers,India Pale Ales,Pale Ales,Pale Lagers,Porters,Specialty Beer,Stouts,Strong Ales,Weird Cocktail,Wheat Beers,Wild/Sour Beers
user_name,,,,,,,,,,,,,,,
-Rick,0,0,0,0,0,14,4,0,1,0,16,7,0,1,7
1000lbgrizzly,0,3,2,0,1,10,10,3,1,2,9,2,0,5,2
1424IpA,0,1,0,0,0,14,9,2,6,1,8,4,0,1,4
2Xmd,0,0,0,1,1,14,8,4,1,3,5,7,1,3,2
2ellas,0,3,1,2,0,15,3,10,3,1,6,3,0,2,1


In [330]:
user_to_cat_first["first_50_cluster"] = our_clustering_4.labels_
user_to_cat_last["first_50_cluster"] = our_clustering_4.labels_

user_to_cat_first.head(5)

category,Bocks,Brown Ales,Dark Ales,Dark Lagers,Hybrid Beers,India Pale Ales,Pale Ales,Pale Lagers,Porters,Specialty Beer,Stouts,Strong Ales,Weird Cocktail,Wheat Beers,Wild/Sour Beers,first_50_cluster
user_name,,,,,,,,,,,,,,,,
-Rick,0,2,0,2,0,29,7,2,2,2,3,1,0,0,0,0
1000lbgrizzly,0,5,0,1,0,4,9,7,2,8,6,4,0,3,1,3
1424IpA,0,0,1,0,0,21,3,1,1,1,12,4,0,1,5,0
2Xmd,0,2,1,1,0,25,5,1,1,2,9,1,0,2,0,0
2ellas,0,2,1,0,0,31,9,0,0,0,2,4,0,1,0,0


In [331]:
# cluster_team = user_to_cat_first['first_50_cluster'].unique()
# categ_beer = user_to_cat_first.columns[1:16]

# for team in cluster_team:
#     for cate in categ_beer:
#        _, ttest_pval = ttest_rel(user_to_cat_first[user_to_cat_first['first_50_cluster'] == team][cate],
#                                  user_to_cat_last[user_to_cat_last['first_50_cluster'] == team][cate])
#        ci_low_mean_first_rating, ci_high_mean_first_rating = bootstrap((user_to_cat_first[user_to_cat_first['first_50_cluster'] == team][cate].values,), np.mean).confidence_interval
#        #ci_low_mean_last_rating, ci_high_mean_last_rating = bootstrap(user_to_cat_last[user_to_cat_last['first_50_cluster']==team][cate], np.mean).confidence_interval
#        if ttest_pval<0.05:
#         print('the p-value for ttest in the team',team,'for the category',cate,'is',ttest_pval)

In [332]:
## Cluster bootstrap

cluster_list=[]
cate_list=[]
ci_low_mean_first_rating_list=[]
ci_high_mean_first_rating_list=[]
mean_first_rating_list=[]
ci_low_mean_last_rating_list=[]
ci_high_mean_last_rating_list=[]
mean_last_rating_list=[]
t_test_list=[]

cluster_team = user_to_cat_first['first_50_cluster'].unique()
categ_beer = user_to_cat_first.columns[0:15]

for team in cluster_team:

    current_team_first_rating = user_to_cat_first[user_to_cat_first['first_50_cluster']==team]
    current_team_last_rating = user_to_cat_last[user_to_cat_last['first_50_cluster']==team]

    for cate in categ_beer:

        current_ttest=ttest_rel(current_team_first_rating[cate],current_team_last_rating[cate])[1]
        #if current_ttest<0.05:
            #print('the p-value for ttest in the team',team,'for the category',cate,'is',current_ttest)

        ci_low_mean_first_rating, ci_high_mean_first_rating = bootstrap((current_team_first_rating[cate].values,), np.mean).confidence_interval
        ci_low_mean_last_rating, ci_high_mean_last_rating = bootstrap((current_team_last_rating[cate].values,), np.mean).confidence_interval

        mean_first_rating_list.append(current_team_first_rating[cate].mean())
        mean_last_rating_list.append(current_team_last_rating[cate].mean())

        cluster_list.append(team)
        cate_list.append(cate)

        ci_low_mean_first_rating_list.append(ci_low_mean_first_rating)
        ci_high_mean_first_rating_list.append(ci_high_mean_first_rating)
        ci_low_mean_last_rating_list.append(ci_low_mean_last_rating)
        ci_high_mean_last_rating_list.append(ci_high_mean_last_rating)
        t_test_list.append(current_ttest)

In [333]:
conf_inter_cluster = pd.DataFrame({'Cluster' : cluster_list, 'Category' : cate_list, 'CI_low_first_rating':ci_low_mean_first_rating_list,
                                'CI_high_first_rating':ci_high_mean_first_rating_list,'mean_first_ratings':mean_first_rating_list,
                                'CI_low_last_rating':ci_low_mean_last_rating_list,'CI_high_last_rating':ci_high_mean_last_rating_list,
                                'mean_last_ratings':mean_last_rating_list,'t_test':t_test_list})

In [334]:
## All bootstrap

cate_list=[]
ci_low_mean_first_rating_list=[]
ci_high_mean_first_rating_list=[]
mean_first_rating_list=[]
ci_low_mean_last_rating_list=[]
ci_high_mean_last_rating_list=[]
mean_last_rating_list=[]
t_test_list=[]
categ_beer = user_to_cat_first.columns[0:15]

for cate in categ_beer:

    current_ttest=ttest_rel(user_to_cat_first[cate],user_to_cat_last[cate])[1]
    #if current_ttest<0.05:
    #print('the p-value for ttest in the team',team,'for the category',cate,'is',current_ttest)
    ci_low_mean_first_rating, ci_high_mean_first_rating = bootstrap((user_to_cat_first[cate].values,), np.mean).confidence_interval
    ci_low_mean_last_rating, ci_high_mean_last_rating = bootstrap((user_to_cat_last[cate].values,), np.mean).confidence_interval
    mean_first_rating_list.append(user_to_cat_first[cate].mean())
    mean_last_rating_list.append(user_to_cat_last[cate].mean())
    cate_list.append(cate)
    ci_low_mean_first_rating_list.append(ci_low_mean_first_rating)
    ci_high_mean_first_rating_list.append(ci_high_mean_first_rating)
    ci_low_mean_last_rating_list.append(ci_low_mean_last_rating)
    ci_high_mean_last_rating_list.append(ci_high_mean_last_rating)
    t_test_list.append(current_ttest)

In [335]:
conf_inter_all= pd.DataFrame({ 'Category' : cate_list, 'CI_low_first_rating':ci_low_mean_first_rating_list,
                    'CI_high_first_rating':ci_high_mean_first_rating_list,'mean_first_ratings':mean_first_rating_list,
                    'CI_low_last_rating':ci_low_mean_last_rating_list,'CI_high_last_rating':ci_high_mean_last_rating_list,
                    'mean_last_ratings':mean_last_rating_list,'t_test':t_test_list})
conf_inter_all['Cluster'] = 'All'


In [336]:
conf_inter_all

,Category,CI_low_first_rating,CI_high_first_rating,mean_first_ratings,CI_low_last_rating,CI_high_last_rating,mean_last_ratings,t_test,Cluster
0,Bocks,1.159794,1.340206,1.247423,0.852577,1.020619,0.932990,1.040049e-08,All
1,Brown Ales,2.536082,2.792784,2.661856,1.976289,2.194845,2.082474,9.232355e-14,All
2,Dark Ales,1.138144,1.314433,1.224742,0.763918,0.916495,0.837113,5.993624e-13,All
3,Dark Lagers,1.271134,1.475258,1.371134,0.723711,0.861856,0.790722,8.970276e-25,All
4,Hybrid Beers,0.271134,0.343299,0.307216,0.277320,0.348454,0.312371,8.332273e-01,All
5,India Pale Ales,10.916495,11.946168,11.427835,11.624742,12.510309,12.076289,1.467942e-02,All
6,Pale Ales,7.520619,7.982474,7.752577,8.164519,8.649485,8.402062,1.559042e-05,All
7,Pale Lagers,4.014433,4.636082,4.309278,2.696907,3.083131,2.882474,2.939323e-18,All
8,Porters,1.998969,2.239175,2.118557,2.102062,2.322680,2.210309,2.196372e-01,All
9,Specialty Beer,1.378643,1.609605,1.486598,1.417526,1.641237,1.525773,6.030390e-01,All


In [337]:
df_conf_inter = pd.concat([conf_inter_all, conf_inter_cluster], ignore_index = True)
#conf_inter.reset_index(drop=True, inplace=True)


In [338]:
df_conf_inter

,Category,CI_low_first_rating,CI_high_first_rating,mean_first_ratings,CI_low_last_rating,CI_high_last_rating,mean_last_ratings,t_test,Cluster
0,Bocks,1.159794,1.340206,1.247423,0.852577,1.020619,0.932990,1.040049e-08,All
1,Brown Ales,2.536082,2.792784,2.661856,1.976289,2.194845,2.082474,9.232355e-14,All
2,Dark Ales,1.138144,1.314433,1.224742,0.763918,0.916495,0.837113,5.993624e-13,All
3,Dark Lagers,1.271134,1.475258,1.371134,0.723711,0.861856,0.790722,8.970276e-25,All
4,Hybrid Beers,0.271134,0.343299,0.307216,0.277320,0.348454,0.312371,8.332273e-01,All
...,...,...,...,...,...,...,...,...,...
70,Stouts,3.092199,3.971631,3.510638,4.744681,5.964539,5.297872,3.360124e-06,2
71,Strong Ales,3.567376,4.347518,3.950355,4.198582,5.212766,4.680851,2.355251e-02,2
72,Weird Cocktail,0.099291,0.219858,0.156028,0.007092,0.056738,0.028369,1.789359e-04,2
73,Wheat Beers,4.283688,5.241135,4.744681,2.534245,3.283688,2.893617,2.932158e-09,2


In [339]:
df_pivot = df_conf_inter.pivot(index='Category', columns='Cluster')
df_pivot.columns = df_pivot.columns.to_flat_index()
df_pivot

,"(CI_low_first_rating, 0)","(CI_low_first_rating, 1)","(CI_low_first_rating, 2)","(CI_low_first_rating, 3)","(CI_low_first_rating, All)","(CI_high_first_rating, 0)","(CI_high_first_rating, 1)","(CI_high_first_rating, 2)","(CI_high_first_rating, 3)","(CI_high_first_rating, All)",...,"(mean_last_ratings, 0)","(mean_last_ratings, 1)","(mean_last_ratings, 2)","(mean_last_ratings, 3)","(mean_last_ratings, All)","(t_test, 0)","(t_test, 1)","(t_test, 2)","(t_test, 3)","(t_test, All)"
Category,,,,,,,,,,,,,,,,,,,,,
Bocks,0.493724,0.668367,1.524823,1.510152,1.159794,0.711297,0.964286,2.085106,1.817259,1.340206,...,0.531381,0.551020,1.631206,1.116751,0.932990,3.781030e-01,2.001985e-03,4.189340e-01,1.210879e-08,1.040049e-08
Brown Ales,1.548117,2.224781,2.397163,3.073604,2.536082,1.924686,2.806122,3.070922,3.484772,2.792784,...,1.263598,1.877551,2.617021,2.489848,2.082474,1.907517e-04,3.737509e-04,6.209248e-01,1.567179e-09,9.232355e-14
Dark Ales,0.506276,0.790816,1.198582,1.507614,1.138144,0.736402,1.142857,1.687943,1.812183,1.314433,...,0.451883,0.602041,1.219858,1.050761,0.837113,2.379335e-02,5.679719e-04,2.220434e-01,3.901109e-10,5.993624e-13
Dark Lagers,0.439331,0.500000,3.035461,1.390863,1.271134,0.665272,0.775510,3.673759,1.675127,1.475258,...,0.489540,0.545918,1.269504,0.923858,0.790722,4.246688e-01,3.469893e-01,1.775291e-21,4.049361e-12,8.970276e-25
Hybrid Beers,0.171548,0.209184,0.276596,0.279188,0.271134,0.301255,0.357143,0.496454,0.393401,0.343299,...,0.246862,0.244898,0.297872,0.390863,0.312371,7.907089e-01,4.381056e-01,2.219053e-01,1.923013e-01,8.332273e-01
India Pale Ales,22.167364,9.066327,2.858156,7.761421,10.916495,23.535565,10.642857,3.815603,8.598985,11.946168,...,16.163180,11.474490,10.148936,10.586294,12.076289,4.386729e-28,2.410343e-03,1.859514e-24,2.487439e-14,1.467942e-02
Pale Ales,7.058577,4.392857,7.219858,9.063452,7.520619,7.916318,5.107143,8.354610,9.761421,7.982474,...,8.142259,7.040816,9.510638,8.840102,8.402062,3.506138e-02,7.210528e-14,7.017638e-06,1.398808e-02,1.559042e-05
Pale Lagers,1.769874,1.107143,12.756798,3.614213,4.014433,2.292887,1.500000,14.652482,4.124365,4.636082,...,2.200837,1.637755,4.652482,3.281726,2.882474,3.107862e-01,4.069503e-02,4.205912e-33,1.749773e-03,2.939323e-18
Porters,1.188285,2.791439,1.322409,2.126904,1.998969,1.523013,3.438776,1.822695,2.479695,2.239175,...,1.962343,2.576531,2.163121,2.195431,2.210309,8.027743e-06,3.266419e-03,3.062677e-03,3.815884e-01,2.196372e-01


In [340]:
df_pivot.columns

Index([     ('CI_low_first_rating', 0),      ('CI_low_first_rating', 1),
            ('CI_low_first_rating', 2),      ('CI_low_first_rating', 3),
        ('CI_low_first_rating', 'All'),     ('CI_high_first_rating', 0),
           ('CI_high_first_rating', 1),     ('CI_high_first_rating', 2),
           ('CI_high_first_rating', 3), ('CI_high_first_rating', 'All'),
             ('mean_first_ratings', 0),       ('mean_first_ratings', 1),
             ('mean_first_ratings', 2),       ('mean_first_ratings', 3),
         ('mean_first_ratings', 'All'),       ('CI_low_last_rating', 0),
             ('CI_low_last_rating', 1),       ('CI_low_last_rating', 2),
             ('CI_low_last_rating', 3),   ('CI_low_last_rating', 'All'),
            ('CI_high_last_rating', 0),      ('CI_high_last_rating', 1),
            ('CI_high_last_rating', 2),      ('CI_high_last_rating', 3),
        ('CI_high_last_rating', 'All'),        ('mean_last_ratings', 0),
              ('mean_last_ratings', 1),        ('me

In [341]:
df_means = df_pivot.iloc[:,10:15]

In [342]:
# mean_user = pd.DataFrame()

# #Cluster 0

# cluster_0 = user_to_cat_first[user_to_cat_first["first_50_cluster"] == 0] #/!\ the number will vary depending on the Kmeans init
# cluster_0.drop(columns = "first_50_cluster", inplace = True)
# #add the average number of beer rated during the FIRST 50 rarings of the cluster 0
# mean_user["first_50_0"] = cluster_0.mean()
# mean_user = mean_user.reset_index(level=0) #makes the category a column rather than the index
# #add the average number of beer rated during the LAST 50 rarings of the cluster 0
# mean_user["last_50_0"] = user_to_cat_last[user_to_cat_last.index.isin(cluster_0.index)].mean().values

# #Cluster 1
# cluster_1 = user_to_cat_first[user_to_cat_first["first_50_cluster"] == 1]#/!\ the number will vary depending on the Kmeans init
# cluster_1.drop(columns = "first_50_cluster", inplace = True)
# #add the average number of beer rated during the FIRST 50 rarings of the cluster 1
# #print(stout_lovers.mean())
# mean_user["first_50_1"] = cluster_1.mean().values
# #add the average number of beer rated during the LAST 50 rarings of the cluster 1
# mean_user["last_50_1"] =  user_to_cat_last[user_to_cat_last.index.isin(cluster_1.index)].mean().values

# #Cluster 2
# cluster_2 = user_to_cat_first[user_to_cat_first["first_50_cluster"] == 2]#/!\ the number will vary depending on the Kmeans init
# cluster_2.drop(columns = "first_50_cluster", inplace = True)
# #add the average number of beer rated during the FIRST 50 ratings of the cluster 2
# mean_user["first_50_2"] = cluster_2.mean().values
# #add the average number of beer rated during the LAST 50 rarings of the cluster 2
# mean_user["last_50_2"] =  user_to_cat_last[user_to_cat_last.index.isin(cluster_2.index)].mean().values

# #Cluster 3
# cluster_3 = user_to_cat_first[user_to_cat_first["first_50_cluster"] == 3]#/!\ the number will vary depending on the Kmeans init
# cluster_3.drop(columns = "first_50_cluster", inplace = True)
# #add the average number of beer rated during the FIRST 50 ratings of the cluster 3
# mean_user["first_50_3"] = cluster_3.mean().values
# #add the average number of beer rated during the LAST 50 rarings of the cluster 3
# mean_user["last_50_3"] =  user_to_cat_last[user_to_cat_last.index.isin(cluster_3.index)].mean().values
# mean_user

# #ALL TOGETHER
# mean_user["all_first_50"] = mean_user[["first_50_0", "first_50_1", "first_50_2", "first_50_3"]].mean(axis=1)
# mean_user["all_last_50"] = mean_user[["last_50_0", "last_50_1", "last_50_2", "last_50_3"]].mean(axis=1)

In [343]:
ipa_col = (df_means.loc['India Pale Ales',:]).argmax()
stout_col = (df_means.loc['Stouts',:]).argmax()
pale_col = (df_means.loc['Pale Lagers',:]).argmax()

possible_indices = [0,1,2,3] #column indices
expl_col = sum(possible_indices) - (ipa_col + stout_col + pale_col)

mapping = {'All' : 4, 'Jean-Michel IPA' : ipa_col, 'Marcel Stout' : stout_col, 
           'James Bland' : pale_col, 'Ada the explorer' : expl_col}


In [344]:
mapping

{'All': 4,
 'Jean-Michel IPA': 0,
 'Marcel Stout': 1,
 'James Bland': 2,
 'Ada the explorer': 3}

In [345]:
#le vrai - redoo avec CI
mean_user = df_pivot
# plotly setup 
fig = go.Figure()

# Add one ore more traces
x = mean_user.index

for i in mapping.keys():
    viz = False
    if i == "All":
        viz = True
    
    #last
    #mean
    fig.add_trace(go.Scatter(x=x, y=mean_user.iloc[:, 25 + mapping[i]].values,
                             name='last 50 beers', mode='lines', line_color='rgb(239,85,59)', 
                             visible=viz, showlegend=True))
    #CI low
    fig.add_trace(go.Scatter(x=x, y=mean_user.iloc[:, 15 + mapping[i]].values,
                             name='first, CI Low', fill=None, mode='lines', line_width = 0.00,
                             fillcolor='rgba(239,85,59,0.3)', visible=viz, showlegend=False))
    #CI high
    fig.add_trace(go.Scatter(x=x, y=mean_user.iloc[:, 20 + mapping[i]].values,
                             name='first, CI high', fill='tonexty', mode='lines', line_width = 0.00,
                             fillcolor='rgba(239,85,59,0.3)', visible=viz, showlegend=False))

    #first
    #mean
    fig.add_trace(go.Scatter(x=x, y=mean_user.iloc[:, 10 + mapping[i]].values,
                             name='first 50 beers', mode='lines', line_color = "rgb(0,204,150)",
                             visible=viz, showlegend=True))
    #CI low
    fig.add_trace(go.Scatter(x=x, y=mean_user.iloc[:, 0 + mapping[i]].values,
                             name='first, CI Low', fill=None, mode='lines', line_width = 0.00,
                             fillcolor='rgba(0,204,150,0.3)', visible=viz, showlegend=False))
    #CI high
    fig.add_trace(go.Scatter(x=x, y=mean_user.iloc[:, 5 + mapping[i]].values,
                             name='first, CI High', fill='tonexty', mode='lines', line_width = 0.00,
                             fillcolor='rgba(0,204,150,0.3)', visible=viz, showlegend=False))

#build 5 mask coresponding to each button
list_0 = [True]*6 + [False]*24
list_1 = [False]*6 + [True]*6 + [False]*18
list_2 = [False]*12 + [True]*6 + [False]*12
list_3 = [False]*18 + [True]*6 + [False]*6
list_4 = [False]*24 + [True]*6

fig.update_layout(
    yaxis_range=[0,25],
    width=1000,
    height=600,
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=1,
            y=1.1,
            buttons=list([
                dict(label="All togheter",
                     method="update",
                     args=[{"visible": list_0},
                           {'title':'All together : average number of beers rated from each category'}]),
                dict(label="Jean-Michel IPA",
                     method="update",
                     args=[{"visible": list_1},
                           {'title' : '"Jean-Michel IPA" team : average number of beers rated from each category'}]),
                dict(label="Marcel Stout",
                     method="update",
                     args=[{"visible": list_2},
                           {'title' : '"Marcel Stout" team : average number of beers rated from each category'}]),
                dict(label="James Bland",
                     method="update",
                     args=[{"visible": list_3},
                           {'title' : '"James Bland" team : average number of beers rated from each category'}]),
                dict(label="Ada the explorer",
                     method="update",
                     args=[{"visible": list_4},
                           {'title' : '"Ada the explorer" team : average number of beers rated from each category'}])

            ]),
        )
    ])

# Set title
fig.update_xaxes(title_text="Category")
fig.update_yaxes(title_text="Number of beers")

fig.update_layout({'title' : 'All together : average number of beers rated from each category'})

fig.show()
fig.write_html("all_first_vs_last.html")